In [1]:
import numpy as np 
import pandas as pd 
import xgboost as xgb
from sklearn.model_selection import KFold, RepeatedKFold
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import log_loss

In [2]:
#验证集
val_dat = np.load('val_dat.npy') #样本量300
val_dat_y = np.load('val_dat_y.npy')

#训练集和测试集
train_best = np.load('X_train.npy')
test_best = np.load('X_test.npy')
y_train = np.load('y_train.npy')

In [3]:
##### xgb
xgb_params = {'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}

folds = KFold(n_splits=5, shuffle=True, random_state=2018)
# oof_xgb = np.zeros(len(train_best))
# predictions_xgb = np.zeros(len(val_dat))
oof_xgb = np.zeros(1500)
predictions_xgb = np.zeros(len(test_best))
target_val = np.zeros(1500)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_best, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(train_best[trn_idx], y_train[trn_idx])
    #val_data = xgb.DMatrix(train_best[val_idx], y_train[val_idx])
    val_dat_xgb = xgb.DMatrix(val_dat,val_dat_y)
    
    watchlist = [(trn_data, 'train'), (val_dat_xgb, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params)
    oof_xgb[range((fold_-1)*300,fold_*300)] = clf.predict(val_dat_xgb, ntree_limit=clf.best_ntree_limit)
    #验证集5次xgb预测结果
    target_val[range((fold_-1)*300,fold_*300)] = val_dat_y
    #测试集xgb预测结果
    predictions_xgb += clf.predict(xgb.DMatrix(test_best), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, target_val)))

fold n°1
[0]	train-rmse:0.422991	valid_data-rmse:0.425381
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.257096	valid_data-rmse:0.259376
[200]	train-rmse:0.156708	valid_data-rmse:0.158766
[300]	train-rmse:0.096156	valid_data-rmse:0.098075
[400]	train-rmse:0.059638	valid_data-rmse:0.0614
[500]	train-rmse:0.037634	valid_data-rmse:0.039221
[600]	train-rmse:0.024404	valid_data-rmse:0.025862
[700]	train-rmse:0.016415	valid_data-rmse:0.017872
[800]	train-rmse:0.011648	valid_data-rmse:0.013162
[900]	train-rmse:0.008801	valid_data-rmse:0.010408
[1000]	train-rmse:0.00709	valid_data-rmse:0.008779
[1100]	train-rmse:0.006043	valid_data-rmse:0.007817
[1200]	train-rmse:0.005336	valid_data-rmse:0.007172
[1300]	train-rmse:0.004798	valid_data-rmse:0.006717
[1400]	train-rmse:0.004378	valid_data-rmse:0.006381
[1500]	train-rmse:0.004024	valid_data-rmse:0.006133
[1600]	train-rmse:

fold n°4
[0]	train-rmse:0.422968	valid_data-rmse:0.425379
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.257033	valid_data-rmse:0.259234
[200]	train-rmse:0.156644	valid_data-rmse:0.158644
[300]	train-rmse:0.096058	valid_data-rmse:0.09799
[400]	train-rmse:0.059572	valid_data-rmse:0.061299
[500]	train-rmse:0.03761	valid_data-rmse:0.039018
[600]	train-rmse:0.02437	valid_data-rmse:0.0255
[700]	train-rmse:0.01642	valid_data-rmse:0.017395
[800]	train-rmse:0.011642	valid_data-rmse:0.012581
[900]	train-rmse:0.008781	valid_data-rmse:0.00979
[1000]	train-rmse:0.007018	valid_data-rmse:0.008137
[1100]	train-rmse:0.005896	valid_data-rmse:0.007153
[1200]	train-rmse:0.005129	valid_data-rmse:0.006514
[1300]	train-rmse:0.004578	valid_data-rmse:0.006094
[1400]	train-rmse:0.004128	valid_data-rmse:0.005786
[1500]	train-rmse:0.003762	valid_data-rmse:0.005567
[1600]	train-rmse:0.00

In [4]:
pd.DataFrame(predictions_xgb)

,0
0,0.907495
1,0.883384
2,0.933309
3,0.903586
4,0.928329
5,0.927922
6,0.935541
7,0.901964
8,0.936906
9,0.928790
